<a id='ксодержанию'></a>
# Содержание
* [Импорт библиотек](#Импортбиблиотек)
* [Загрузка данных](#Загрузкаданных)
* [Модели](#Модели)
  + [raw_desc_tfidf_Ridge](#raw_desc_tfidf_Ridge)
    - [Моделируем](#Моделируемraw_desc_tfidf_Ridge)
    - [Отчёт](#Отчётraw_desc_tfidf_Ridge)
  + [raw_desc_tfidf_Ridge_merged_df](#raw_desc_tfidf_Ridge_merged_df)
    - [Моделируем](#Моделируемraw_desc_tfidf_Ridge_merged_df)
    - [Отчёт](#Отчётraw_desc_tfidf_Ridge_merged_df)
  + [raw_desc_tfidf_Ridge_only_Russia](#raw_desc_tfidf_Ridge_only_Russia)
    - [Моделируем](#Моделируемraw_desc_tfidf_Ridge_only_Russia)
    - [Отчёт](#Отчётraw_desc_tfidf_Ridge_only_Russia)
  + [lem_desc_resp_con_tfidf_Ridge_10_Rus](#lem_desc_resp_con_tfidf_Ridge_10_Rus)
    - [Моделируем](#Моделируемlem_desc_resp_con_tfidf_Ridge_10_Rus)
    - [Отчёт](#Отчётlem_desc_resp_con_tfidf_Ridge_10_Rus)
  + [lem_desc_emb213_Ridge](#lem_desc_emb213_Ridge)
    - [Моделируем](#Моделируемlem_desc_emb213_Ridge)
    - [Отчёт](#Отчётlem_desc_emb213_Ridge)
  + [lem_desc_emb213_Ridge](#lem_desc_emb213_Ridge)
    - [Моделируем](#Моделируемlem_desc_emb213_Ridge)
    - [Отчёт](#Отчётlem_desc_emb213_Ridge)
  + [lem_desc_emb213_Ridge_ruandRus](#lem_desc_emb213_Ridge_ruandRus)
    - [Моделируем](#Моделируемlem_desc_emb213_Ridge_ruandRus)
    - [Отчёт](#Отчётlem_desc_emb213_Ridge_ruandRus)   
  + [lem_desc_emb214_Ridge_Rus](#lem_desc_emb214_Ridge_Rus)
    - [Моделируем](#Моделируемlem_desc_emb214_Ridge_Rus)
    - [Отчёт](#Отчётlem_desc_emb214_Ridge_Rus)     
  + [lem_desc_emb187_Ridge_Rus](#lem_desc_emb187_Ridge_Rus)
    - [Моделируем](#Моделируемlem_desc_emb187_Ridge_Rus)
    - [Отчёт](#Отчётlem_desc_emb187_Ridge_Rus)     
  + [raw_desc_deeppavlov_Ridge](#raw_desc_deeppavlov_Ridge)
    - [Моделируем](#Моделируемraw_desc_deeppavlov_Ridge)
    - [Отчёт](#Отчётraw_desc_deeppavlov_Ridge)    
  + [raw_desc_sberbank-ai_sbert_large_nlu_ru_Ridge](#raw_desc_sberbank-ai_sbert_large_nlu_ru_Ridge)
    - [Моделируем](#Моделируемraw_desc_sberbank-ai_sbert_large_nlu_ru_Ridge)
    - [Отчёт](#Отчётraw_desc_sberbank-ai_sbert_large_nlu_ru_Ridge)     

<a id='Импортбиблиотек'></a> 
# Импорт библиотек
[к содержанию](#ксодержанию)

In [34]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from sklearn.metrics import mean_absolute_error
import numpy as np
import sys, os
import pickle
import scipy.sparse as sp
import zipfile
import wget
import gensim
import numpy as np
from collections import defaultdict

<a id='Загрузкаданных'></a>
# Загрузка данных
[к содержанию](#ксодержанию)

In [3]:
!gdown --id 1A-59GMNuJWbrbCc17zvBPkOycSdCBdCA
!gdown --id 1G5OXqdq9rQo7XWqdAk2FO3uXbQiymNw0
df = pd.read_csv('vacancies_lemmatized.csv')
df2 = pd.read_csv('hhparser_vacancy.csv')
df2['raw_description'] = df2['description']
df2.loc[(df2['salary_gross'] == False), 'salary_from'] = df2['salary_from'] / 0.87
df = df[df['language'] == 'ru']

Downloading...
From: https://drive.google.com/uc?id=1A-59GMNuJWbrbCc17zvBPkOycSdCBdCA
To: /content/vacancies_lemmatized.csv
100% 902M/902M [00:09<00:00, 94.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1G5OXqdq9rQo7XWqdAk2FO3uXbQiymNw0
To: /content/hhparser_vacancy.csv
100% 313M/313M [00:04<00:00, 75.1MB/s]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1021: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [3]:
sys.path.append(os.path.abspath('../'))
from src.models import utils

In [8]:
utils.salary_process(df)

C:\Users\Lisa\MSU_diploma\src\models\utils.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['salary_from'][df['salary_currency'] == curr] = df['salary_from'][df['salary_currency'] == curr].apply(lambda x: x*currency[curr])


<a id='Модели'></a>
# Модели
[к содержанию](#ксодержанию)

<a id='raw_desc_tfidf_Ridge'></a>
## raw_desc_tfidf_Ridge
[к содержанию](#ксодержанию)

<a id='Моделируемraw_desc_tfidf_Ridge'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [4]:
X = df[['raw_description', 'salary_from']].dropna()
X_train, X_test, y_train, y_test = train_test_split(X[['raw_description']], X['salary_from'], test_size=0.2, random_state=42)

In [5]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=14000)
tfidf.fit(X_train['raw_description'])
features = tfidf.transform(X_train['raw_description'])

In [6]:
# Сохраняем tfidf.vocabulary_
pickle.dump(tfidf.vocabulary_,open("raw_desc_tfidf_14000.pkl","wb"))

In [7]:
del X_train

In [8]:
model = Ridge()
model.fit(features, y_train)

Ridge()

In [9]:
del features

In [10]:
features_test = tfidf.transform(X_test['raw_description'])

In [11]:
y_pred = model.predict(features_test)

In [12]:
# Сохраняем модель и y_pred
joblib.dump(model, 'raw_desc_tfidf_Ridge.pkl', compress=8)
joblib.dump(y_pred, 'y_pred_raw_desc_tfidf_Ridge.pkl')

['y_pred_raw_desc_tfidf_Ridge_prof_area.pkl']

<a id='Отчётraw_desc_tfidf_Ridge'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [14]:
regression_report, salary = utils.report_regression(df[['salary_from', 'prof_area']].dropna(), y_test, y_pred, 'prof_area')

In [32]:
regression_report_raw = pd.DataFrame(regression_report)
regression_report_raw.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
regression_report_raw.index = np.append(salary['true'].unique(), 'Total')
regression_report_raw.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
Административный персонал,"14,686.0","34,107.2","19,094.3"
"Транспорт, логистика","17,231.7","56,083.3","36,173.1"
Продажи,"16,777.2","36,663.7","94,863.2"
"Строительство, недвижимость","20,637.8","69,788.8","45,062.1"
"Управление персоналом, тренинги","18,351.2","45,731.5","39,318.6"
Безопасность,"12,430.5","42,114.3","22,491.0"
"Информационные технологии, интернет, телеком","24,713.6","50,753.6","47,597.3"
"Производство, сельское хозяйство","23,030.9","52,506.1","75,003.3"
"Медицина, фармацевтика","11,908.5","49,268.2","27,663.4"
"Банки, инвестиции, лизинг","14,746.0","43,680.9","23,496.3"


<a id='raw_desc_tfidf_Ridge_merged_df'></a>
## raw_desc_tfidf_Ridge_merged_df
[к содержанию](#ксодержанию)

<a id='Моделируемraw_desc_tfidf_Ridge_merged_df'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [33]:
df3 = pd.concat([df, df2])
df3 = df3.drop_duplicates(subset='id', keep="first")

In [34]:
X = df3[['raw_description', 'salary_from']].dropna()
X_train, X_test, y_train, y_test = train_test_split(X[['raw_description']], X['salary_from'], test_size=0.2, random_state=42)

In [35]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=14000)
tfidf.fit(X_train['raw_description'])
features = tfidf.transform(X_train['raw_description'])

In [36]:
# Сохраняем tfidf.vocabulary_
pickle.dump(tfidf.vocabulary_,open("raw_desc_tfidf_14000_merged_df.pkl","wb"))

In [37]:
del X_train

In [38]:
model = Ridge()
model.fit(features, y_train)

Ridge()

In [39]:
del features

In [40]:
features_test = tfidf.transform(X_test['raw_description'])

In [41]:
y_pred = model.predict(features_test)

In [42]:
# Сохраняем модель и y_pred
joblib.dump(model, 'raw_desc_tfidf_Ridge_merged_df.pkl', compress=8)
joblib.dump(y_pred, 'y_pred_raw_desc_tfidf_Ridge_merged_df.pkl')

['y_pred_raw_desc_tfidf_Ridge_prof_area_merged_df.pkl']

<a id='Отчётraw_desc_tfidf_Ridge_merged_df'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [43]:
regression_report = pd.DataFrame()
regression_report['MAE'] = [round(mean_absolute_error(y_pred, y_test), 1)]
regression_report['salary_from_mean'] = [round(df3['salary_from'].mean(), 1)]
regression_report['salary_from_std'] = [round(df3['salary_from'].std(), 1)]

In [44]:
regression_report

,MAE,salary_from_mean,salary_from_std
0,24628.1,56563.6,76708.8


<a id='raw_desc_tfidf_Ridge_only_Russia'></a>
## raw_desc_tfidf_Ridge_only_Russia
[к содержанию](#ксодержанию)

<a id='Моделируемraw_desc_tfidf_Ridge_only_Russia'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [46]:
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[df['country_name'] == 'Россия']
utils.salary_process(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [47]:
X = df[['raw_description', 'salary_from']].dropna()
X_train, X_test, y_train, y_test = train_test_split(X[['raw_description']], X['salary_from'], test_size=0.2, random_state=42)

In [48]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=14000)
tfidf.fit(X_train['raw_description'])
features = tfidf.transform(X_train['raw_description'])

In [ ]:
# Сохраняем tfidf.vocabulary_
pickle.dump(tfidf.vocabulary_,open("raw_desc_tfidf_14000_merged_df.pkl","wb"))

In [49]:
del X_train

In [50]:
model = Ridge()
model.fit(features, y_train)

Ridge()

In [51]:
del features

In [52]:
features_test = tfidf.transform(X_test['raw_description'])

In [53]:
y_pred = model.predict(features_test)

In [58]:
# Сохраняем модель и y_pred
joblib.dump(model, 'raw_desc_tfidf_Ridge_only_Russia.pkl', compress=8)
joblib.dump(y_pred, 'y_pred_raw_desc_tfidf_Ridge_only_Russia.pkl')

['y_pred_raw_desc_tfidf_Ridge_prof_area_only_Russia.pkl']

<a id='Отчётraw_desc_tfidf_Ridge_only_Russia'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [56]:
regression_report, salary = utils.report_regression(df[['salary_from', 'prof_area']].dropna(), y_test, y_pred, 'prof_area')

In [57]:
regression_report_raw = pd.DataFrame(regression_report)
regression_report_raw.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
regression_report_raw.index = np.append(salary['true'].unique(), 'Total')
regression_report_raw.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
Продажи,"11,309.2","38,688.7","26,803.7"
"Производство, сельское хозяйство","18,225.5","56,875.1","81,732.8"
"Транспорт, логистика","15,889.0","59,084.2","36,609.6"
"Медицина, фармацевтика","8,920.6","50,545.3","27,492.8"
"Информационные технологии, интернет, телеком","18,267.0","47,960.1","35,573.2"
Рабочий персонал,"12,621.9","54,111.8","32,314.8"
"Начало карьеры, студенты","12,975.9","36,702.1","23,053.2"
"Строительство, недвижимость","19,456.4","74,235.5","45,836.4"
"Туризм, гостиницы, рестораны","13,146.5","38,022.2","24,052.2"
"Спортивные клубы, фитнес, салоны красоты","12,418.2","34,212.3","17,309.7"


<a id='lem_desc_resp_con_tfidf_Ridge_10_Rus'></a>
## lem_desc_resp_con_tfidf_Ridge_10_Rus
[к содержанию](#ксодержанию)

<a id='Моделируемlem_desc_resp_con_tfidf_Ridge_10_Rus'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [4]:
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[df['country_name'] == 'Россия']
utils.salary_process(df)

C:\Users\Lisa\MSU_diploma\src\models\utils.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['salary_from'][df['salary_currency'] == curr] = df['salary_from'][df['salary_currency'] == curr].apply(lambda x: x*currency[curr])


In [8]:
X = df[['raw_description', 'salary_from', 'lem_desc_resp', 'lem_desc_con']].dropna()
X_train, X_test, y_train, y_test = train_test_split(X[['raw_description', 'lem_desc_resp', 'lem_desc_con']], X['salary_from'], test_size=0.2, random_state=42)

In [9]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=10000)
tfidf.fit(X_train['raw_description'])
features = tfidf.transform(X_train['raw_description'])

In [10]:
tfidf2 = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=10000)
tfidf2.fit(X_train['lem_desc_resp'])
features2 = tfidf2.transform(X_train['lem_desc_resp'])

tfidf3 = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words={'russian', 'english'}, max_features=10000)
tfidf3.fit(X_train['lem_desc_con'])
features3 = tfidf3.transform(X_train['lem_desc_con'])

In [15]:
# Сохраняем tfidf.vocabulary_
pickle.dump(tfidf.vocabulary_,open("../models/lem_desc_resp_con_tfidf_Ridge_10_Rus/lem_desc_tfidf.pkl","wb"))
pickle.dump(tfidf2.vocabulary_,open("../models/lem_desc_resp_con_tfidf_Ridge_10_Rus/lem_desc_resp_tfidf.pkl","wb"))
pickle.dump(tfidf3.vocabulary_,open("../models/lem_desc_resp_con_tfidf_Ridge_10_Ruslem_desc_con_tfidf.pkl","wb"))

In [14]:
features = sp.hstack((features, features2, features3))

In [16]:
del X_train

In [17]:
model = Ridge()
model.fit(features, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [18]:
del features

In [19]:
features_test = tfidf.transform(X_test['raw_description'])

In [20]:
features_test2 = tfidf2.transform(X_test['lem_desc_resp'])
features_test3 = tfidf3.transform(X_test['lem_desc_con'])

In [22]:
features_test = sp.hstack((features_test, features_test2, features_test3))

In [23]:
y_pred = model.predict(features_test)

In [24]:
# Сохраняем модель и y_pred
joblib.dump(model, '../models/lem_desc_resp_con_tfidf_Ridge_10_Rus/model.pkl', compress=8)
joblib.dump(y_pred, '../models/lem_desc_resp_con_tfidf_Ridge_10_Rus/y_pred.pkl')

['../models/lem_desc_resp_con_tfidf_Ridge_10_Rus/y_pred.pkl']

<a id='Отчётlem_desc_resp_con_tfidf_Ridge_10_Rus'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [25]:
regression_report, salary = utils.report_regression(df[['salary_from', 'prof_area']].dropna(), y_test, y_pred, 'prof_area')

In [26]:
regression_report = pd.DataFrame(regression_report)
regression_report.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
regression_report.index = np.append(salary['true'].unique(), 'Total')
regression_report.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
Продажи,"11,825.9","38,688.7","26,803.7"
"Производство, сельское хозяйство","18,063.2","56,875.1","81,732.8"
"Транспорт, логистика","15,986.3","59,084.2","36,609.6"
"Медицина, фармацевтика","8,554.1","50,545.3","27,492.8"
"Информационные технологии, интернет, телеком","19,275.4","47,960.1","35,573.2"
Рабочий персонал,"12,423.2","54,111.8","32,314.8"
"Начало карьеры, студенты","13,190.9","36,702.1","23,053.2"
"Строительство, недвижимость","19,261.0","74,235.5","45,836.4"
"Туризм, гостиницы, рестораны","13,666.7","38,022.2","24,052.2"
"Спортивные клубы, фитнес, салоны красоты","12,323.2","34,212.3","17,309.7"


<a id='lem_desc_emb213_Ridge'></a>
## lem_desc_emb213_Ridge
[к содержанию](#ксодержанию)

<a id='Моделируемlem_desc_emb213_Ridge'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [27]:
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[df['language'] == 'ru']
utils.salary_process(df)
df = df[['lem_raw_desc', 'salary_from', 'prof_area']].dropna()

C:\Users\Lisa\MSU_diploma\src\models\utils.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['salary_from'][df['salary_currency'] == curr] = df['salary_from'][df['salary_currency'] == curr].apply(lambda x: x*currency[curr])


In [30]:
model_url = 'http://vectors.nlpl.eu/repository/20/213.zip'
# m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    model = gensim.models.KeyedVectors.load("./213/model.model")

In [33]:
tfidf = TfidfVectorizer()
tfidf.fit(df['lem_raw_desc'])
max_idf = max(tfidf.idf_)
word2weight = defaultdict(
    lambda: max_idf,
    [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

In [ ]:
w2v = dict(zip(model.index_to_key, model.vectors))
data_mean = utils.mean_vectorizer(w2v).fit().transform(df['lem_raw_desc'], word2weight)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data_mean, df['salary_from'], test_size=0.2, random_state=42)

In [11]:
model = Ridge()
model.fit(X_train, y_train)

Ridge()

In [12]:
y_pred = model.predict(X_test)

In [13]:
# Сохраняем модель и y_pred
joblib.dump(model, '../models/lem_desc_emb213_Ridge/model.pkl', compress=8)
joblib.dump(y_pred, '../models/lem_desc_emb213_Ridge/y_pred.pkl')

['../models/lem_desc_emb213_Ridge/y_pred.pkl']

<a id='Отчётraw_desc_tfidf_Ridge'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [14]:
regression_report, salary = utils.report_regression(df[['salary_from', 'prof_area']].dropna(), y_test, y_pred, 'prof_area')

In [15]:
regression_report_raw = pd.DataFrame(regression_report)
regression_report_raw.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
regression_report_raw.index = np.append(salary['true'].unique(), 'Total')
regression_report_raw.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
Административный персонал,"13,151.2","34,107.2","19,094.3"
"Транспорт, логистика","19,044.8","56,083.3","36,173.1"
Продажи,"13,011.9","36,663.7","94,863.2"
"Строительство, недвижимость","25,079.5","69,788.8","45,062.1"
"Управление персоналом, тренинги","16,656.3","45,731.5","39,318.6"
Безопасность,"14,194.4","42,114.3","22,491.0"
"Информационные технологии, интернет, телеком","25,977.7","50,753.6","47,597.3"
"Производство, сельское хозяйство","25,309.9","52,506.1","75,003.3"
"Медицина, фармацевтика","16,211.2","49,268.2","27,663.4"
"Банки, инвестиции, лизинг","13,707.1","43,680.9","23,496.3"


<a id='lem_desc_emb213_Ridge_ruandRus'></a>
## lem_desc_emb213_Ridge_ruandRus
[к содержанию](#ксодержанию)

<a id='Моделируемlem_desc_emb213_Ridge_ruandRus'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [37]:
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[(df['language'] == 'ru') & (df['country_name'] == 'Россия')]
utils.salary_process(df)
df = df[['lem_raw_desc', 'salary_from', 'prof_area']].dropna()

C:\Users\Lisa\MSU_diploma\src\models\utils.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['salary_from'][df['salary_currency'] == curr] = df['salary_from'][df['salary_currency'] == curr].apply(lambda x: x*currency[curr])


In [35]:
model_url = 'http://vectors.nlpl.eu/repository/20/213.zip'
# m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    model = gensim.models.KeyedVectors.load("./213/model.model")

In [38]:
tfidf = TfidfVectorizer()
tfidf.fit(df['lem_raw_desc'])
max_idf = max(tfidf.idf_)
word2weight = defaultdict(
    lambda: max_idf,
    [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

In [39]:
w2v = dict(zip(model.index_to_key, model.vectors))
data_mean = utils.mean_vectorizer(w2v).fit().transform(df['lem_raw_desc'], word2weight)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(data_mean, df['salary_from'], test_size=0.2, random_state=42)

In [41]:
model = Ridge()
model.fit(X_train, y_train)

Ridge()

In [42]:
y_pred = model.predict(X_test)

In [44]:
# Сохраняем модель и y_pred
joblib.dump(model, '../models/lem_desc_emb213_Ridge_ruandRus/model.pkl', compress=8)
joblib.dump(y_pred, '../models/lem_desc_emb213_Ridge_ruandRus/y_pred.pkl')

['../models/lem_desc_emb213_Ridge_ruandRus/y_pred.pkl']

<a id='Отчётlem_desc_emb213_Ridge_ruandRus'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [45]:
regression_report, salary = utils.report_regression(df[['salary_from', 'prof_area']].dropna(), y_test, y_pred, 'prof_area')

In [46]:
regression_report_raw = pd.DataFrame(regression_report)
regression_report_raw.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
regression_report_raw.index = np.append(salary['true'].unique(), 'Total')
regression_report_raw.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
"Начало карьеры, студенты","16,772.4","36,702.1","23,053.2"
"Производство, сельское хозяйство","22,958.1","56,878.9","81,741.8"
"Медицина, фармацевтика","15,551.3","50,561.5","27,495.8"
"Бухгалтерия, управленческий учет, финансы предприятия","15,690.5","43,982.4","30,217.6"
"Туризм, гостиницы, рестораны","14,297.8","38,031.7","24,060.0"
"Строительство, недвижимость","26,415.4","74,235.5","45,836.4"
Продажи,"13,270.7","38,685.8","26,799.3"
Административный персонал,"13,826.0","35,649.7","18,880.5"
"Управление персоналом, тренинги","17,607.9","47,184.9","41,016.0"
Рабочий персонал,"15,511.5","54,115.0","32,315.9"


<a id='lem_desc_emb214_Ridge_Rus'></a>
## lem_desc_emb214_Ridge_Rus
[к содержанию](#ксодержанию)

<a id='Моделируемlem_desc_emb214_Ridge_Rus'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [3]:
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[df['country_name'] == 'Россия']
utils.salary_process(df)
df = df[['lem_raw_desc', 'salary_from', 'prof_area']].dropna()

C:\Users\Lisa\MSU_diploma\src\models\utils.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['salary_from'][df['salary_currency'] == curr] = df['salary_from'][df['salary_currency'] == curr].apply(lambda x: x*currency[curr])


In [5]:
model_url = 'http://vectors.nlpl.eu/repository/20/214.zip'
# m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    model = gensim.models.KeyedVectors.load("./214/model.model")

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(df['lem_raw_desc'])
max_idf = max(tfidf.idf_)
word2weight = defaultdict(
    lambda: max_idf,
    [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

In [ ]:
w2v = dict(zip(model.index_to_key, model.vectors))
data_mean = utils.mean_vectorizer(w2v).fit().transform(df['lem_raw_desc'], word2weight)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_mean, df['salary_from'], test_size=0.2, random_state=42)

In [ ]:
model = Ridge()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [55]:
# Сохраняем модель и y_pred
joblib.dump(model, '../models/lem_desc_emb214_Ridge_Rus/model.pkl', compress=8)
joblib.dump(y_pred, '../models/lem_desc_emb214_Ridge_Rus/y_pred.pkl')

['../models/lem_desc_emb214_Ridge_Rus/y_pred.pkl']

<a id='Отчётlem_desc_emb214_Ridge_Rus'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [56]:
regression_report, salary = utils.report_regression(df[['salary_from', 'prof_area']].dropna(), y_test, y_pred, 'prof_area')

In [57]:
regression_report_raw = pd.DataFrame(regression_report)
regression_report_raw.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
regression_report_raw.index = np.append(salary['true'].unique(), 'Total')
regression_report_raw.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
Продажи,"12,840.1","38,688.7","26,803.7"
"Производство, сельское хозяйство","21,032.9","56,875.1","81,732.8"
"Транспорт, логистика","18,778.1","59,084.2","36,609.6"
"Медицина, фармацевтика","14,013.9","50,545.3","27,492.8"
"Информационные технологии, интернет, телеком","19,844.4","47,960.1","35,573.2"
Рабочий персонал,"15,926.5","54,111.8","32,314.8"
"Начало карьеры, студенты","15,454.1","36,702.1","23,053.2"
"Строительство, недвижимость","24,494.7","74,235.5","45,836.4"
"Туризм, гостиницы, рестораны","14,136.3","38,022.2","24,052.2"
"Спортивные клубы, фитнес, салоны красоты","12,268.1","34,212.3","17,309.7"


<a id='lem_desc_emb187_Ridge_Rus'></a>
## lem_desc_emb187_Ridge_Rus
[к содержанию](#ксодержанию)

<a id='Моделируемlem_desc_emb187_Ridge_Rus'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [3]:
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[df['country_name'] == 'Россия']
utils.salary_process(df)
df = df[['lem_raw_desc', 'salary_from', 'prof_area']].dropna()

C:\Users\Lisa\MSU_diploma\src\models\utils.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['salary_from'][df['salary_currency'] == curr] = df['salary_from'][df['salary_currency'] == curr].apply(lambda x: x*currency[curr])


In [4]:
model_url = 'http://vectors.nlpl.eu/repository/20/187.zip'
# m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    model = gensim.models.KeyedVectors.load("./187/model.model")

In [5]:
tfidf = TfidfVectorizer()
tfidf.fit(df['lem_raw_desc'])
max_idf = max(tfidf.idf_)
word2weight = defaultdict(
    lambda: max_idf,
    [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

In [6]:
w2v = dict(zip(model.index2word, model.vectors))
data_mean = utils.mean_vectorizer(w2v).fit().transform(df['lem_raw_desc'], word2weight)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data_mean, df['salary_from'], test_size=0.2, random_state=42)

In [8]:
model = Ridge()
model.fit(X_train, y_train)

Ridge()

In [9]:
y_pred = model.predict(X_test)

In [10]:
# Сохраняем модель и y_pred
joblib.dump(model, '../models/lem_desc_emb187_Ridge_Rus/model.pkl', compress=8)
joblib.dump(y_pred, '../models/lem_desc_emb187_Ridge_Rus/y_pred.pkl')

['../models/lem_desc_emb187_Ridge_Rus/y_pred.pkl']

<a id='Отчётlem_desc_emb187_Ridge_Rus'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [11]:
regression_report, salary = utils.report_regression(df[['salary_from', 'prof_area']].dropna(), y_test, y_pred, 'prof_area')

In [12]:
regression_report_raw = pd.DataFrame(regression_report)
regression_report_raw.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
regression_report_raw.index = np.append(salary['true'].unique(), 'Total')
regression_report_raw.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
Продажи,"12,941.3","39,105.2","26,803.7"
"Производство, сельское хозяйство","21,306.7","54,537.4","81,732.8"
"Транспорт, логистика","19,918.0","59,566.2","36,609.6"
"Медицина, фармацевтика","14,192.4","49,936.5","27,492.8"
"Информационные технологии, интернет, телеком","19,052.2","48,127.8","35,573.2"
Рабочий персонал,"16,663.5","53,907.9","32,314.8"
"Начало карьеры, студенты","16,188.2","36,367.1","23,053.2"
"Строительство, недвижимость","25,122.3","74,208.0","45,836.4"
"Туризм, гостиницы, рестораны","15,322.8","37,059.8","24,052.2"
"Спортивные клубы, фитнес, салоны красоты","13,151.8","34,746.5","17,309.7"


<a id='raw_desc_deeppavlov_Ridge'></a>
## raw_desc_deeppavlov_Ridge
[к содержанию](#ксодержанию)

<a id='Моделируемraw_desc_deeppavlov_Ridge'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [8]:
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[df['language'] == 'ru']
utils.salary_process(df)

In [ ]:
transformer = SentenceTransformer('DeepPavlov/rubert-base-cased-sentence')
X = transformer.encode(df['raw_description'].values)
joblib.dump(X, '../models/bert/X_DeepPavlov_rubert-base-cased-sentence.pkl', compress=9)

In [24]:
X = joblib.load('../models/bert/X_DeepPavlov_rubert-base-cased-sentence.pkl')

In [25]:
X = pd.DataFrame(X)
X['salary_from'] = df['salary_from']

In [28]:
X = X.dropna()
X_train, X_test, y_train, y_test = train_test_split(X.loc[:,:767], X['salary_from'], test_size=0.2, random_state=42)

In [29]:
model = Ridge()
model.fit(X_train, y_train)

Ridge()

In [30]:
y_pred = model.predict(X_test)

In [31]:
# Сохраняем модель и y_pred
joblib.dump(model, '../models/raw_desc_deeppavlov_Ridge/model.pkl', compress=9)
joblib.dump(y_pred, '../models/raw_desc_deeppavlov_Ridge/y_pred.pkl', compress=9)

['../models/raw_desc_deeppavlov_Ridge/y_pred.pkl']

<a id='Отчётraw_desc_deeppavlov_Ridge'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [32]:
regression_report, salary = utils.report_regression(df[['salary_from', 'prof_area']].dropna(), y_test, y_pred, 'prof_area')

In [33]:
regression_report = pd.DataFrame(regression_report)
regression_report.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
regression_report.index = np.append(salary['true'].unique(), 'Total')
regression_report.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
Продажи,"19,037.3","34,944.9","94,863.2"
"Строительство, недвижимость","35,042.7","71,369.2","45,062.1"
"Банки, инвестиции, лизинг","16,958.4","42,852.2","23,496.3"
Административный персонал,"18,806.9","33,753.9","19,094.3"
Рабочий персонал,"24,026.6","49,406.1","31,799.0"
"Наука, образование","22,412.6","29,456.3","19,463.1"
"Транспорт, логистика","24,963.5","55,000.4","36,173.1"
"Бухгалтерия, управленческий учет, финансы предприятия","17,270.8","40,872.9","28,182.5"
"Медицина, фармацевтика","20,618.8","47,889.8","27,663.4"
Страхование,"13,262.4","37,088.9","11,627.0"


<a id='raw_desc_sberbank-ai_sbert_large_nlu_ru_Ridge'></a>
## raw_desc_sberbank-ai_sbert_large_nlu_ru_Ridge
[к содержанию](#ксодержанию)

<a id='Моделируемraw_desc_sberbank-ai_sbert_large_nlu_ru_Ridge'></a>
### Моделируем
[к содержанию](#ксодержанию)

In [8]:
df = pd.read_csv('vacancies_lemmatized.csv')
df = df[df['language'] == 'ru']
utils.salary_process(df)

In [ ]:
X = transformer.encode(df['raw_description'].values)
joblib.dump(X, '../models/bert/X_sberbank-ai_sbert_large_nlu_ru.pkl', compress=9)

In [5]:
X = joblib.load('../models/bert/X_sberbank-ai_sbert_large_nlu_ru.pkl')

In [12]:
X = pd.DataFrame(X)
X['salary_from'] = df['salary_from']

In [19]:
X = X.dropna()
X_train, X_test, y_train, y_test = train_test_split(X.loc[:,:1023], X['salary_from'], test_size=0.2, random_state=42)

In [20]:
model = Ridge()
model.fit(X_train, y_train)

Ridge()

In [21]:
y_pred = model.predict(X_test)

In [24]:
# Сохраняем модель и y_pred
joblib.dump(model, '../models/raw_desc_deeppavlov_Ridge/model.pkl', compress=9)
joblib.dump(y_pred, '../models/raw_desc_deeppavlov_Ridge/y_pred.pkl', compress=9)

['../models/lem_desc_resp_con_tfidf_Ridge_10_Rus/y_pred.pkl']

<a id='Отчётraw_desc_sberbank-ai_sbert_large_nlu_ru_Ridge'></a>
### Отчёт
[к содержанию](#ксодержанию)

In [22]:
regression_report, salary = utils.report_regression(df[['salary_from', 'prof_area']].dropna(), y_test, y_pred, 'prof_area')

In [23]:
regression_report = pd.DataFrame(regression_report)
regression_report.columns = ['MAE', 'salary_from_mean', 'salary_from_std']
regression_report.index = np.append(salary['true'].unique(), 'Total')
regression_report.style.format(formatter='{:,.1f}').background_gradient()

,MAE,salary_from_mean,salary_from_std
Продажи,"19,363.8","34,944.9","94,863.2"
"Строительство, недвижимость","35,392.4","71,369.2","45,062.1"
"Банки, инвестиции, лизинг","17,828.4","42,852.2","23,496.3"
Административный персонал,"19,300.1","33,753.9","19,094.3"
Рабочий персонал,"24,352.2","49,406.1","31,799.0"
"Наука, образование","22,188.6","29,456.3","19,463.1"
"Транспорт, логистика","25,414.8","55,000.4","36,173.1"
"Бухгалтерия, управленческий учет, финансы предприятия","17,390.8","40,872.9","28,182.5"
"Медицина, фармацевтика","21,123.4","47,889.8","27,663.4"
Страхование,"12,360.2","37,088.9","11,627.0"
